In [12]:
# pip install pyzotero

In [13]:
from pyzotero import zotero
import os
import tweepy as tw
import pandas as pd
import datetime
import json, sys
from datetime import date, timedelta  
import datetime
import plotly.express as px

In [14]:
library_id = '2514686'
library_type = 'group'
api_key = '' # api_key is only needed for private groups and libraries
zot = zotero.Zotero(library_id, library_type)


# All items in the Zotero Intelligence bibliography library

In [15]:
items = zot.everything(zot.top())

data3=[]
columns3=['Title','Publication type', 'Link to publication', 'Abstract', 'Zotero link', 'Date published', 'FirstName2', 'Publisher', 'Journal']

for item in items:
    data3.append((
        item['data']['title'], 
        item['data']['itemType'], 
        item['data']['url'], 
        item['data']['abstractNote'], 
        item['links']['alternate']['href'],
        item['data'].get('date'),
        item['data']['creators'],
        item['data'].get('publisher'),
        item['data'].get('publicationTitle')
        )) 
pd.set_option('display.max_colwidth', None)

df = pd.DataFrame(data3, columns=columns3)

df['Publication type'] = df['Publication type'].replace(['thesis'], 'Thesis')
df['Publication type'] = df['Publication type'].replace(['journalArticle'], 'Journal article')
df['Publication type'] = df['Publication type'].replace(['book'], 'Book')
df['Publication type'] = df['Publication type'].replace(['bookSection'], 'Book chapter')
df['Publication type'] = df['Publication type'].replace(['blogPost'], 'Blog post')
df['Publication type'] = df['Publication type'].replace(['videoRecording'], 'Video')
df['Publication type'] = df['Publication type'].replace(['podcast'], 'Podcast')
df['Publication type'] = df['Publication type'].replace(['magazineArticle'], 'Magazine article')
df['Publication type'] = df['Publication type'].replace(['webpage'], 'Webpage')
df['Publication type'] = df['Publication type'].replace(['newspaperArticle'], 'Newspaper article')
df['Publication type'] = df['Publication type'].replace(['report'], 'Report')
df['Publication type'] = df['Publication type'].replace(['forumPost'], 'Forum post')
df['Publication type'] = df['Publication type'].replace(['manuscript'], 'Manuscript')
df['Publication type'] = df['Publication type'].replace(['document'], 'Document')
df['Publication type'] = df['Publication type'].replace(['forumPost'], 'Forum post')
df['Publication type'] = df['Publication type'].replace(['conferencePaper'], 'Conference paper')
df['Publication type'] = df['Publication type'].replace(['film'], 'Film')
df['Publication type'] = df['Publication type'].replace(['presentation'], 'Presentation')

df['Publisher'] = df['Publisher'].replace(['Taylor & Francis Group', 'Taylor and Francis', 'Taylor & Francis'], 'Taylor and Francis')
df['Publisher'] = df['Publisher'].replace(['Routledge', 'Routledge Handbooks Online'], 'Routledge')
df['Publisher'] = df['Publisher'].replace(['Praeger Security International', 'Praeger'], 'Praeger')

df['Journal'] = df['Journal'].replace(['International Journal of Intelligence and Counterintelligence', 'International Journal of Intelligence and CounterIntelligence'], 'Intl Journal of Intelligence and Counterintelligence')
df['Journal'] = df['Journal'].replace(['Intelligence and national security', 'Intelligence and National Security', 'Intelligence & National Security'], 'Intelligence and National Security')

In [16]:
day_allitems = datetime.date.today().isoformat()

In [17]:
df_fa = df['FirstName2']
df_fa = pd.DataFrame(df_fa.tolist())
df_fa = df_fa[0]
df_fa = df_fa.apply(lambda x: {} if pd.isna(x) else x)
df_new = pd.json_normalize(df_fa, errors='ignore') 
df = pd.concat([df, df_new], axis=1)
df['firstName'] = df['firstName'].fillna('null')
df['lastName'] = df['lastName'].fillna('null')

In [18]:
df.to_csv('all_items.csv')

In [19]:
import psycopg2
from sqlalchemy import create_engine
import psycopg2.extras as extras